In [1]:
year = "2021"
filing_type = "10-K"

start = year+"-01-01"
end = year+"-12-31"

In [1]:
from main_data import get_data

full_data = get_data(
    "TSLA",
    "2021",
    filing_type="10-Q",
    include_amends=True,
    num_workers=8
)

Started for TSLA


In [2]:
full_data

defaultdict(list,
            {'TSLA': [{'year': '202109',
               'ticker': 'TSLA',
               'all_texts': {'FINANCIAL_STATEMENTS': '',
                'MANAGEMENT_DISCUSSION': "The following discussion and analysis should be read in conjunction with the consolidated financial statements and the related notes included elsewhere in this Quarterly Report on Form 10-Q. Our mission is to accelerate the world’s transition to sustainable energy. We design, develop, manufacture, lease and sell high-performance fully electric vehicles, solar energy generation systems and energy storage products. We also offer maintenance, installation, operation, financial and other services related to our products. In 2021, we have produced 624,582 vehicles and delivered 627,572 vehicles through the third quarter. We are currently focused on increasing vehicle production and capacity, improving and developing battery technologies, improving our FSD and Autopilot capabilities, increasing the affor

In [16]:
# len(full_data)
# full_data

In [ ]:
"\xa0","\n","\t",

In [3]:
for i in full_data['TSLA']:
    print(i['year'])
    print(i['filing_type'])

202109
10-Q
202106
10-Q
202103
10-Q


In [22]:
documents = []
metadata = []

for tic_data in full_data['TSLA']:
    # print(tic_data)
    curr_year = tic_data["year"]
    ticker = tic_data["ticker"]
    filing_type = tic_data["filing_type"]

    for section, section_text in tic_data["all_texts"].items():
        documents.append(section_text)
        metadata.append(
            {
                "year": curr_year,
                "ticker": ticker,
                "section": section,
                "filing_type": filing_type,
            }
        )

In [24]:
metadata

[{'year': '2021',
  'ticker': 'TSLA',
  'section': 'BUSINESS',
  'filing_type': '10-K/A'},
 {'year': '2021',
  'ticker': 'TSLA',
  'section': 'RISK_FACTORS',
  'filing_type': '10-K/A'},
 {'year': '2021',
  'ticker': 'TSLA',
  'section': 'UNRESOLVED_STAFF_COMMENTS',
  'filing_type': '10-K/A'},
 {'year': '2021',
  'ticker': 'TSLA',
  'section': 'PROPERTIES',
  'filing_type': '10-K/A'},
 {'year': '2021',
  'ticker': 'TSLA',
  'section': 'LEGAL_PROCEEDINGS',
  'filing_type': '10-K/A'},
 {'year': '2021',
  'ticker': 'TSLA',
  'section': 'MINE_SAFETY',
  'filing_type': '10-K/A'},
 {'year': '2021',
  'ticker': 'TSLA',
  'section': 'MARKET_FOR_REGISTRANT_COMMON_EQUITY',
  'filing_type': '10-K/A'},
 {'year': '2021',
  'ticker': 'TSLA',
  'section': 'MANAGEMENT_DISCUSSION',
  'filing_type': '10-K/A'},
 {'year': '2021',
  'ticker': 'TSLA',
  'section': 'MARKET_RISK_DISCLOSURES',
  'filing_type': '10-K/A'},
 {'year': '2021',
  'ticker': 'TSLA',
  'section': 'FINANCIAL_STATEMENTS',
  'filing_type':